In [14]:


from py2neo import Graph 
from py2neo import GraphService



In [29]:
with open('password.txt') as f:
    lines = f.readlines()
    
p = lines[0]

gs = GraphService("bolt://localhost:7687", user="neo4j", password=p)
graph = gs.default_graph

In [49]:
def get_counts_word(word) :
    """
    Gets the postive, negative, counts for the given word
    
    """
    node = graph.run("MATCH (n {label: '"+ word+ "'})return n")
    for x in node:
        for y in x:
            return (y['pos_count'],y['neg_count'])

In [48]:
get_counts_word('hello')

('4387', '1061')

In [67]:
def get_sentiment_sentence(sentence) :
    """
    Calculates Sentiment of Sentence by averaging for each word: Positive / Total , then the average for 
    all words in sentence is the sentiment.
    
    """
    average_sentiment_words = []
    for x in sentence.split(): 
        word_tup = get_counts_word(x.lower())
        if word_tup is None : 
            print(x + ' is not in the graph')
        else :
            word_positive = float(word_tup[0]) / (float(word_tup[0]) + float(word_tup[1]))
            print(word_positive)
            average_sentiment_words.append(word_positive)
    return sum(average_sentiment_words) / len(average_sentiment_words)

In [65]:
get_sentiment_sentence('hello alex my name is steve')

0.8052496328928047
0.5832438238453276
0.39796649460735767
0.653811169728943
0.45931883087718106
0.5546623794212219


0.5757087218954726

In [66]:
get_sentiment_sentence('I hate you')

0.384589256862147
0.13000408496732027
0.6517001210801684


0.3887644876365452

In [106]:
def get_next_word(last_word,is_positive) :
    """
    Given the last word and whether the sentence is positive, predicts the next word.
    
    
    """
    
    if is_positive == 1 :
        most_positive = 0
        most_positive_word = ''
        
        node = graph.run("MATCH (n {label: '"+ last_word+ "'}) - [r] -> (x) return x order by r.Positive desc limit 1")
        for x in node:
            for y in x:
                return y['label']
    else :
        node = graph.run("MATCH (n {label: '"+ last_word+ "'}) - [r] -> (x) return x order by r.Negative desc limit 1")
        for x in node:
            for y in x:
                return y['label']

In [107]:
get_next_word('my',1)

'puppy'

In [108]:
get_next_word('my',0)

'soul'

In [109]:
def user_input() :
    sentence = input("Enter Sentence: ")
    sentence_words = sentence.split()
    last_word = sentence_words[-1]
    print(last_word)
    
    #get next positive word
    if get_sentiment_sentence(sentence) > 0.5 :
        predicted_word = get_next_word(last_word,1)
    else :
        predicted_word = get_next_word(last_word,0)
    
    print(predicted_word)
    print(sentence + ' ' + str(predicted_word))

In [110]:
user_input()

Enter Sentence: hello I am 
am
0.8052496328928047
0.384589256862147
0.4278129053177691
ready
hello I am  ready
